In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import BertModel
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
import string
import pandas as pd

In [ ]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df.head()

In [ ]:
#remove html tags
def remove_html_tag(text): 
    return re.sub(
        '<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', 
        '', text)
df['clean_text']=df.review.apply(remove_html_tag)

In [ ]:
x=list(df['clean_text'])


In [ ]:
y=list(df['sentiment'])


In [ ]:
y=list(pd.get_dummies(y,drop_first=True)['positive'])


In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings=tokenizer(xtrain,truncation=True,padding=True)
test_encodings=tokenizer(xtest,truncation=True,padding=True)

In [ ]:
import tensorflow as tf
train_dataset=tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    ytrain
))
test_dataset=tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    ytest
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
training_args=TFTrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=2,              # total number of training epochs
        per_device_train_batch_size=8,  # batch size per device during training
        per_device_eval_batch_size=16,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
    )

In [ ]:
with training_args.strategy.scope():
    model=TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
trainer=TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset)
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)[1].shape

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
output

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(ytest,output)
cm

In [ ]:
trainer.save_model('senti_model')